In [ ]:
import csv
import pandas as pd
import numpy as np
import seaborn as sns

from pydrive.auth import GoogleAuth
from google.colab import drive
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier  # Importing RandomForestClassifier


In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

train_file_id = '1J6pVdmBbEN7kvAq6f36s9Nx7qFNVpjfD'
test_file_id = '1NnH_ZI6EbISZJO1YOLfmLs9ex11wO2AH'

train_download = drive.CreateFile({'id': train_file_id})
test_download = drive.CreateFile({'id': test_file_id})

# Download the file to a local disc
train_download.GetContentFile('train_file.csv')
test_download.GetContentFile('test_file.csv')


# Specify the data type for the problematic column (e.g., as 'str' if it should be a string)
# If you're not sure about the correct data type, you can use 'str' to read everything as strings and then process it later.
dtype_dict = {6: 'str'}

train_data = pd.read_csv("train_file.csv", dtype=dtype_dict, low_memory=False)
test_data = pd.read_csv("test_file.csv", dtype=dtype_dict, low_memory=False)


In [ ]:
import pandas as pd
# Display basic information about the dataset
train_data_info = train_data.info()
train_data_head = train_data.head()

train_data_info, train_data_head
# Checking for missing values and getting basic statistics
missing_values = train_data.isnull().sum()
basic_stats = train_data.describe(include='all')
target_distribution = train_data['Transported'].value_counts(normalize=True)

missing_values, basic_stats, target_distribution
# Function to split the 'Cabin' column into 'Deck', 'Num', and 'Side'
def split_cabin(dataframe):
    # Splitting the 'Cabin' column
    cabin_split = dataframe['Cabin'].str.split('/', expand=True)
    cabin_split.columns = ['Deck', 'Num', 'Side']

    # Concatenating the new columns with the original dataframe
    return pd.concat([dataframe.drop(columns=['Cabin']), cabin_split], axis=1)

# Apply the function to both training and testing datasets
train_df = split_cabin(train_data)
test_df = split_cabin(test_data)

# Display the first few rows of the modified training data
train_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Deck,Num,Side
0,0001_01,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False,B,0,P
1,0002_01,Earth,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True,F,0,S
2,0003_01,Europa,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False,A,0,S
3,0003_02,Europa,False,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False,A,0,S
4,0004_01,Earth,False,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True,F,1,S


In [ ]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Assuming train_df and test_df are defined elsewhere in your code
# For example:
# train_df = pd.read_csv('path_to_train_data.csv')
# test_df = pd.read_csv('path_to_test_data.csv')

# Identifying categorical and numerical columns
categorical_cols = ['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'Deck', 'Side']
numerical_cols = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

# Pipeline for imputing and encoding categorical columns
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Pipeline for imputing numerical columns
numerical_transformer = SimpleImputer(strategy='median')

# Combining transformers into a ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Applying the transformations to the training data
X_train = preprocessor.fit_transform(train_df.drop(['PassengerId', 'Name', 'Transported'], axis=1))
y_train = train_df['Transported']

# Applying the same transformations to the test data (excluding target variable)
X_test = preprocessor.transform(test_df.drop(['PassengerId', 'Name'], axis=1))

# Getting categorical feature names after one-hot encoding
categorical_features = preprocessor.named_transformers_['cat']['onehot'].get_feature_names_out(input_features=categorical_cols)

# Combine categorical feature names with numerical feature names
all_feature_names = np.concatenate([numerical_cols, categorical_features])

# Checking the shape of the processed data
print(X_train.shape, X_test.shape, y_train.shape)

# Printing all feature names
print(all_feature_names)


(8693, 26) (4277, 26) (8693,)
['Age' 'RoomService' 'FoodCourt' 'ShoppingMall' 'Spa' 'VRDeck'
 'HomePlanet_Earth' 'HomePlanet_Europa' 'HomePlanet_Mars'
 'CryoSleep_False' 'CryoSleep_True' 'Destination_55 Cancri e'
 'Destination_PSO J318.5-22' 'Destination_TRAPPIST-1e' 'VIP_False'
 'VIP_True' 'Deck_A' 'Deck_B' 'Deck_C' 'Deck_D' 'Deck_E' 'Deck_F' 'Deck_G'
 'Deck_T' 'Side_P' 'Side_S']


In [ ]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Assuming train_df and test_df are defined elsewhere in your code
# train_df = pd.read_csv('path_to_train_data.csv')
# test_df = pd.read_csv('path_to_test_data.csv')

def preprocess_data(train_df, test_df):
    # Identifying categorical and numerical columns
    categorical_cols = ['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'Deck', 'Side']
    numerical_cols = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

    # Pipeline for imputing and encoding categorical columns
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

    # Pipeline for imputing and scaling numerical columns
    numerical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])

    # Combining transformers into a ColumnTransformer
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, numerical_cols),
            ('cat', categorical_transformer, categorical_cols)
        ])

    # Applying the transformations to the training data
    X_train = preprocessor.fit_transform(train_df.drop(['PassengerId', 'Name', 'Transported'], axis=1))
    y_train = train_df['Transported']

    # Applying the same transformations to the test data (excluding target variable)
    X_test = preprocessor.transform(test_df.drop(['PassengerId', 'Name'], axis=1))

    # Getting categorical feature names after one-hot encoding
    categorical_features = preprocessor.named_transformers_['cat']['onehot'].get_feature_names_out(input_features=categorical_cols)

    # Combine categorical feature names with numerical feature names
    all_feature_names = np.concatenate([numerical_cols, categorical_features])

    return X_train, y_train, X_test, all_feature_names

# Preprocess the data
X_train, y_train, X_test, all_feature_names = preprocess_data(train_df, test_df)

# Checking the shape of the processed data
print(X_train.shape, X_test.shape, y_train.shape)

# Printing all feature names
print(all_feature_names)


(8693, 26) (4277, 26) (8693,)
['Age' 'RoomService' 'FoodCourt' 'ShoppingMall' 'Spa' 'VRDeck'
 'HomePlanet_Earth' 'HomePlanet_Europa' 'HomePlanet_Mars'
 'CryoSleep_False' 'CryoSleep_True' 'Destination_55 Cancri e'
 'Destination_PSO J318.5-22' 'Destination_TRAPPIST-1e' 'VIP_False'
 'VIP_True' 'Deck_A' 'Deck_B' 'Deck_C' 'Deck_D' 'Deck_E' 'Deck_F' 'Deck_G'
 'Deck_T' 'Side_P' 'Side_S']


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Initializing the Random Forest Classifier
random_forest_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Performing cross-validation to evaluate the model
cv_scores = cross_val_score(random_forest_model, X_train, y_train, cv=5, scoring='accuracy')

# Calculating the average cross-validation score
average_cv_score = cv_scores.mean()

average_cv_score


0.7870702644470488

In [ ]:
# Training the model on the entire training dataset
random_forest_model.fit(X_train, y_train)

# Making predictions on the test dataset
test_predictions = random_forest_model.predict(X_test)

# Preparing the submission file
submission_df = pd.DataFrame({'PassengerId': test_df['PassengerId'], 'Transported': test_predictions})
submission_df.head()

# Save the dataframe to a CSV file
submission_df.to_csv('submission.csv', index=False)

from google.colab import files
files.download('submission.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Training the model on the entire training dataset
random_forest_model.fit(X_train, y_train)

# Making predictions on the test dataset
test_predictions = random_forest_model.predict(X_test)

# Preparing the submission file
submission_df = pd.DataFrame({'PassengerId': test_df['PassengerId'], 'Transported': test_predictions})
submission_df.head()

# Save the dataframe to a CSV file
submission_df.to_csv('submission.csv', index=False)

from google.colab import files
files.download('submission.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Handling missing values
imputer = SimpleImputer(strategy='mean')
train_data_filled = pd.DataFrame(imputer.fit_transform(train_data.select_dtypes(include=['float64'])))
train_data_filled.columns = train_data.select_dtypes(include=['float64']).columns
train_data_filled.index = train_data.index

# Encoding categorical variables
train_data_encoded = pd.get_dummies(train_data.select_dtypes(include=['object', 'bool']))

# Combining numerical and categorical data
train_data_processed = pd.concat([train_data_filled, train_data_encoded], axis=1)

# Repeat the same for test_data


In [ ]:
# Identify features and target variable
X = train_data_processed.drop(['Transported'], axis=1)  # Assuming 'Transported' is the target variable
y = train_data_processed['Transported']


In [ ]:
# Splitting the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Decision Tree
decision_tree_model = DecisionTreeClassifier(random_state=42)
decision_tree_model.fit(X_train, y_train)

# Random Forest
random_forest_model = RandomForestClassifier(random_state=42)
random_forest_model.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [ ]:
# Evaluating Decision Tree
y_pred_tree = decision_tree_model.predict(X_val)
print(f"Decision Tree Accuracy: {accuracy_score(y_val, y_pred_tree)}")

# Evaluating Random Forest
y_pred_forest = random_forest_model.predict(X_val)
print(f"Random Forest Accuracy: {accuracy_score(y_val, y_pred_forest)}")


Decision Tree Accuracy: 0.7745830937320299
Random Forest Accuracy: 0.7751581368602645
